# Likelihood Stats
Kai Sandbrink

2024-09-30

This script aggregates the likelihood tests for both task 1 and task 2. Note that it requires re-running the networks on the human data to determine the likelihood of the trajectories.

## Task 1
### Likelihood estimation: NOTE: This section requires re-running likelihood fits from NNs and will not work out of the box with the included data

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import pickle
from utils import format_axis

import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from utils import flatten

analysis_folder = os.path.join('panels', 'fig_task1')
os.makedirs(analysis_folder, exist_ok=True)

effs_to_plot = [1, 0.5, 0]
n_steps = 50
smoothing_window = 8
ylim = (0, 0.3)
human_data_file_base = '..' # make sure this points to the repo root folder, or adjust file names in get_clean_data() function

In [ ]:
from human_utils_project import get_clean_data, sort_train_test

day = 'day2'
exp_date = '24-01-22-29'

day1_test_mask_cutoff = {
    "groupA": {"lower": 10, "upper": 90},
    "groupB": {"lower": 8, "upper": 72}
}

group = None

df, effs_train, effs_test, test_start = get_clean_data(day = int(day[-1]), exp_date = exp_date, day1_test_mask_cutoff=day1_test_mask_cutoff, group=group, file_base=human_data_file_base)

cmap_humans = mpl.colormaps['Greens']

In [ ]:
## NOTE: These behavioral files need to be re-generated to determine likelihoods

df_file_A_22 = '/home/kai/Documents/Projects/HumanObserveBetEfficacyAnalysis/results/behavior/20240527162302_behavior_diff_effs_24-01-22_day2_with_nets_groupA.pkl'
df_file_B_22 = '/home/kai/Documents/Projects/HumanObserveBetEfficacyAnalysis/results/behavior/20240527162823_behavior_diff_effs_24-01-22_day2B_with_nets_groupB.pkl'

df_file_A_29 = '/home/kai/Documents/Projects/HumanObserveBetEfficacyAnalysis/results/behavior/20240527164021_behavior_diff_effs_24-01-29_day2_with_nets_groupA.pkl'
df_file_B_29 = '/home/kai/Documents/Projects/HumanObserveBetEfficacyAnalysis/results/behavior/20240527164530_behavior_diff_effs_24-01-29_day2B_with_nets_groupB.pkl'

df_A = pd.read_pickle(df_file_A_22)
df_B = pd.read_pickle(df_file_B_22)

df_A_29 = pd.read_pickle(df_file_A_29)
df_B_29 = pd.read_pickle(df_file_B_29)

df_A = pd.concat([df_A, df_A_29])
df_B = pd.concat([df_B, df_B_29])

## only keep the rows in df_A that match an index in df (keeping in mind there might be missing keys)
df_A = df_A[df_A.index.isin(df.index)]
df_B = df_B[df_B.index.isin(df.index)]

In [ ]:
from settings_ana import pepe_human_ape_models as ape_models
from settings_ana import pepe_human_control_models as control_models
from human_utils_project import sort_train_test


test_liks_A_ape = []
test_liks_A_noape = []
test_liks_B_ape = []
test_liks_B_noape = []

for model in ape_models:
    train_liks_A, test_liks_A = sort_train_test(df_A['step_l_%d' %model], df_A['effs'], test_start=5)
    test_liks_A_ape.append(test_liks_A)

for model in control_models:
    train_liks_A, test_liks_A = sort_train_test(df_A['step_l_%d' %model], df_A['effs'], test_start=5)
    test_liks_A_noape.append(test_liks_A)

for model in ape_models:
    train_liks_B, test_liks_B = sort_train_test(df_B['step_l_%d' %model], df_B['effs'], test_start=4)
    test_liks_B_ape.append(test_liks_B)

for model in control_models:
    train_liks_B, test_liks_B = sort_train_test(df_B['step_l_%d' %model], df_B['effs'], test_start=4)
    test_liks_B_noape.append(test_liks_B)

test_liks_A_ape = np.array(test_liks_A_ape)
test_liks_A_noape = np.array(test_liks_A_noape)
test_liks_B_ape = np.array(test_liks_B_ape)
test_liks_B_noape = np.array(test_liks_B_noape)

test_log_liks_A_ape = np.log(test_liks_A_ape).sum(axis=(2,3)).mean(axis=0)
test_log_liks_A_noape = np.log(test_liks_A_noape).sum(axis=(2,3)).mean(axis=0)
test_log_liks_B_ape = np.log(test_liks_B_ape).sum(axis=(2,3)).mean(axis=0)
test_log_liks_B_noape = np.log(test_liks_B_noape).sum(axis=(2,3)).mean(axis=0)

test_log_liks_ape = np.concatenate([test_log_liks_A_ape, test_log_liks_B_ape])
test_log_liks_noape = np.concatenate([test_log_liks_A_noape, test_log_liks_B_noape])

In [ ]:
test_log_liks_ape

array([ -363.28003,  -607.51385,  -241.255  ,  -576.93225,  -453.8461 ,
        -804.7565 ,  -385.7108 ,  -182.80573,  -758.4735 ,  -750.74695,
        -201.57501,  -618.94006,  -574.3519 ,  -638.19324,  -554.9235 ,
        -716.6028 ,  -976.5804 ,  -235.1971 ,  -353.23267,  -187.86916,
        -311.7671 ,  -580.4729 ,  -282.10562,  -208.50418,  -297.87756,
        -309.13483,  -216.8081 ,  -254.10506,  -500.02383,  -535.4814 ,
        -192.32184,  -260.59918,  -452.52158,  -246.2952 ,  -320.43243,
        -296.84903,  -771.46063,  -323.11627,  -234.5413 ,  -494.2418 ,
        -314.53595,  -227.60286,  -612.5376 ,  -285.7924 ,  -247.99092,
        -587.6279 ,  -914.4038 ,  -330.02673,  -965.02325,  -231.38913,
        -774.7299 ,  -264.68396,  -681.96436,  -637.4237 ,  -422.69614,
        -448.85825,  -389.39325,  -331.95258,  -363.09833, -1091.1127 ,
        -578.2069 ,  -842.6576 ,  -536.635  ,  -713.7173 ,  -416.23984,
        -799.1179 ,  -305.7055 ,  -256.49286,  -328.69858,  -737

In [ ]:
from groupBMC import GroupBMC

L = np.array([test_log_liks_ape, test_log_liks_noape])
result = GroupBMC(L).get_result()

print("Model frequencies", result.frequency_mean)
print("Model frequencies variance", result.frequency_var)
print("Model frequences std", np.sqrt(result.frequency_var))
print("Model SEM", np.sqrt(result.frequency_var)/ np.sqrt(len(test_log_liks_ape)))
print("N=", len(test_log_liks_ape))
print("Exceedance probabilities", result.exceedance_probability)

Model frequencies [0.99553571 0.00446429]
Model frequencies variance [3.93305829e-05 3.93305829e-05]
Model frequences std [0.00627141 0.00627141]
Model SEM [0.00059526 0.00059526]
N= 111
Exceedance probabilities [1.00000000e+00 2.16065456e-37]


In [ ]:
from scipy.stats import ttest_rel
from statsmodels.stats.multitest import multipletests


## MEANS AND SEMS
print("Overall N", len(df))

p_values = []

for level, liks, liks_noape in zip(plotted_levels, test_liks_ape, test_liks_noape):
    ## AVERAGE STATS
    print("Test Controllability ", level)
    print("Mean APE", liks.mean())
    print("StdErr APE", liks.std()/np.sqrt(liks.shape[0]))
    print("N APE", liks.shape[0])
    
    print("Mean NOAPE", liks_noape.mean())
    print("StdErr NOAPE", liks_noape.std()/np.sqrt(liks_noape.shape[0]))
    print("N NOAPE", liks_noape.shape[0])

    ## paired t-test
    ttest = ttest_rel(liks, liks_noape, alternative='greater')
    print(ttest)
    p_values.append(ttest.pvalue)

## MULTIPLE COMPARISONS
rejects, p_values_corrected, _, _ = multipletests(p_values, alpha=0.05, method='holm')
print(rejects)
print(p_values_corrected)

Overall N 111
Test Controllability  1
Mean APE 0.64828455
StdErr APE 0.032885752642098774
N APE 46
Mean NOAPE 0.5193321
StdErr NOAPE 0.01781263220435191
N NOAPE 46
TtestResult(statistic=4.382374171570158, pvalue=3.478475123286644e-05, df=45)
Test Controllability  0.5
Mean APE 0.6098049
StdErr APE 0.023917397474663916
N APE 65
Mean NOAPE 0.4813526
StdErr NOAPE 0.013706170910942134
N NOAPE 65
TtestResult(statistic=5.2869110429825845, pvalue=8.028168688747074e-07, df=64)
Test Controllability  0
Mean APE 0.5624307
StdErr APE 0.0219809138864937
N APE 46
Mean NOAPE 0.5342141
StdErr NOAPE 0.02146576568873027
N NOAPE 46
TtestResult(statistic=1.0590984795602612, pvalue=0.1476034607207268, df=45)
[ True  True False]
[6.95695025e-05 2.40845061e-06 1.47603461e-01]


In [ ]:
import pandas as pd

# Initialize a list to store the results
results = []

for level, liks, liks_noape in zip(plotted_levels, test_liks_ape, test_liks_noape):
    # Calculate statistics
    mean_ape = liks.mean()
    stderr_ape = liks.std()/np.sqrt(liks.shape[0])
    n_ape = liks.shape[0]
    
    mean_noape = liks_noape.mean()
    stderr_noape = liks_noape.std()/np.sqrt(liks_noape.shape[0])
    n_noape = liks_noape.shape[0]

    # Perform t-test
    ttest = ttest_rel(liks, liks_noape, alternative='greater')

    # Calculate degrees of freedom
    df = len(liks) - 1

    # Append results to the list
    results.append([level, n_ape, mean_ape, stderr_ape, mean_noape, stderr_noape, df, ttest.pvalue, ])

# Convert the list to a DataFrame
df_results = pd.DataFrame(results, columns=['Controllability', 'N', 'Mean APE', 'StdErr APE', 'Mean Standard', 'StdErr Standard', 'df', 'p-value',])

# Perform multiple comparisons correction
rejects, p_values_corrected, _, _ = multipletests(df_results['p-value'], alpha=0.05, method='holm')

# Add the corrected p-values and rejection decisions to the DataFrame
df_results['p-value corrected'] = p_values_corrected
df_results['reject'] = rejects

# Set "Controllability" to index column
#df_results.set_index('Controllability', inplace=True, drop=True)
df_results.rename_axis('Index', inplace=True)

df_results = df_results.round(3)

# Print the DataFrame
df_results

,Controllability,N,Mean APE,StdErr APE,Mean Standard,StdErr Standard,df,p-value,p-value corrected,reject
Index,,,,,,,,,,
0,1.0,46,0.648,0.033,0.519,0.018,45,0.000,0.000,True
1,0.5,65,0.610,0.024,0.481,0.014,64,0.000,0.000,True
2,0.0,46,0.562,0.022,0.534,0.021,45,0.148,0.148,False


In [ ]:
import pandas as pd

# Initialize a list to store the results
results = []

#for level, liks, liks_noape in zip(effs, test_liks_ape, test_liks_noape):
for eff in np.arange(0, 1.01, 0.125):
    print(eff, effs_test[0], effs_test[1])
    if eff in effs_test[0]:
        print(effs_test[0].index(eff))
    else:
        print(effs_test[1].index(eff))
    liks = test_liks_A_ape[:, :, effs_test[0].index(eff)] if eff in effs_test[0] else test_liks_B_ape[:, :, effs_test[1].index(eff)]
    liks = liks.mean(axis=(0,-1))
    liks_noape = test_liks_A_noape[:, :, effs_test[0].index(eff)] if eff in effs_test[0] else test_liks_B_noape[:, :, effs_test[1].index(eff)]
    liks_noape = liks_noape.mean(axis=(0,-1))

    # Calculate statistics
    mean_ape = liks.mean()
    stderr_ape = liks.std()/np.sqrt(liks.shape[0])
    n_ape = liks.shape[0]
    
    mean_noape = liks_noape.mean()
    stderr_noape = liks_noape.std()/np.sqrt(liks_noape.shape[0])
    n_noape = liks_noape.shape[0]

    # Perform t-test
    ttest = ttest_rel(liks, liks_noape, alternative='greater')
    print(ttest)

    # Calculate degrees of freedom
    df = len(liks) - 1

    # Append results to the list
    results.append([level, n_ape, mean_ape, stderr_ape, mean_noape, stderr_noape, df, ttest[0], ttest.pvalue, ])

# Convert the list to a DataFrame
df_results = pd.DataFrame(results, columns=['Controllability', 'N', 'Mean APE', 'StdErr APE', 'Mean Standard', 'StdErr Standard', 'df', 't-stat', 'p-value',])

# Perform multiple comparisons correction
rejects, p_values_corrected, _, _ = multipletests(df_results['p-value'], alpha=0.05, method='holm')

# Add the corrected p-values and rejection decisions to the DataFrame
df_results['p-value corrected'] = p_values_corrected
df_results['reject'] = rejects

# Set "Controllability" to index column
#df_results.set_index('Controllability', inplace=True, drop=True)
df_results.rename_axis('Index', inplace=True)

df_results = df_results.round(3)

# Print the DataFrame
df_results

0.0 [0, 0.25, 0.75, 1.0] [0.125, 0.375, 0.5, 0.625, 0.875]
0
TtestResult(statistic=1.0590984795602612, pvalue=0.1476034607207268, df=45)
0.125 [0, 0.25, 0.75, 1.0] [0.125, 0.375, 0.5, 0.625, 0.875]
0
TtestResult(statistic=4.970849309827105, pvalue=2.6348240429590767e-06, df=64)
0.25 [0, 0.25, 0.75, 1.0] [0.125, 0.375, 0.5, 0.625, 0.875]
1
TtestResult(statistic=5.007846162060969, pvalue=4.480918858467041e-06, df=45)
0.375 [0, 0.25, 0.75, 1.0] [0.125, 0.375, 0.5, 0.625, 0.875]
1
TtestResult(statistic=5.861651237543418, pvalue=8.723881005775804e-08, df=64)
0.5 [0, 0.25, 0.75, 1.0] [0.125, 0.375, 0.5, 0.625, 0.875]
2
TtestResult(statistic=5.2869110429825845, pvalue=8.028168688747074e-07, df=64)
0.625 [0, 0.25, 0.75, 1.0] [0.125, 0.375, 0.5, 0.625, 0.875]
3
TtestResult(statistic=5.037902736140775, pvalue=2.052148936299469e-06, df=64)
0.75 [0, 0.25, 0.75, 1.0] [0.125, 0.375, 0.5, 0.625, 0.875]
2
TtestResult(statistic=5.218004463378611, pvalue=2.2184022254776446e-06, df=45)
0.875 [0, 0.25, 0.

,Controllability,N,Mean APE,StdErr APE,Mean Standard,StdErr Standard,df,t-stat,p-value,p-value corrected,reject
Index,,,,,,,,,,,
0,0,46,0.562,0.022,0.534,0.021,45,1.059,0.148,0.148,False
1,0,65,0.591,0.017,0.493,0.015,64,4.971,0.000,0.000,True
2,0,46,0.654,0.021,0.527,0.019,45,5.008,0.000,0.000,True
3,0,65,0.613,0.021,0.480,0.012,64,5.862,0.000,0.000,True
4,0,65,0.610,0.024,0.481,0.014,64,5.287,0.000,0.000,True
5,0,65,0.611,0.027,0.485,0.013,64,5.038,0.000,0.000,True
6,0,46,0.660,0.029,0.488,0.014,45,5.218,0.000,0.000,True
7,0,65,0.592,0.031,0.488,0.012,64,3.310,0.001,0.002,True
8,0,46,0.648,0.033,0.519,0.018,45,4.382,0.000,0.000,True


In [ ]:
#test_liks_A_ape.shape
test_liks_B_ape.shape

(10, 65, 5, 50)

In [ ]:
df_results['p-value']

0    [[0.21268047276849933, 0.09889201639633535, 0....
1    [[0.21268047276849933, 0.9686015378958075, 0.6...
2    [[0.21268047276849933, 0.16316568433457407, 0....
3    [[0.21268047276849933, 0.9209015922552268, 0.1...
4    [[0.21268047276849933, 0.9209015922552268, 0.9...
5    [[0.21268047276849933, 0.9686015378958075, 0.6...
6    [[0.21268047276849933, 0.09889201639633535, 0....
7    [[0.21268047276849933, 0.9686015378958075, 0.0...
8    [[0.21268047276849933, 0.16316568433457407, 0....
Name: p-value, dtype: object

## Task 2
### NOTE: This section requires re-running likelihood fits from NNs and will not work out of the box with the included data

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import pickle
from utils import format_axis

import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from utils import flatten

analysis_folder = os.path.join('panels', 'fig_task2')
os.makedirs(analysis_folder, exist_ok=True)

effs_to_plot = [1, 0.5, 0]
n_steps = 50
smoothing_window = 4
ylim = (-0.01,0.45)

exp_date = '24-01-22-29'
human_data_file_base = '..' # make sure the cwd points to the root repo folder, else adjust here

In [ ]:
from human_utils_project import get_clean_data, sort_train_test

day = 'day3'
exp_date = '24-01-22-29'

day1_test_mask_cutoff = {
    "groupA": {"lower": 10, "upper": 90},
    "groupB": {"lower": 8, "upper": 72}
}

group = None

df, effs_train, effs_test, test_start = get_clean_data(day = int(day[-1]), exp_date = exp_date, day1_test_mask_cutoff=day1_test_mask_cutoff, group=group, file_base=human_data_file_base)

cmap_humans = mpl.colormaps['Greens']

In [ ]:
## NOTE: Likelihoods need to be recomputed for the public datasets

df_file_A_22 = '/home/kai/Documents/Projects/HumanObserveBetEfficacyAnalysis/results/behavior/20240527162546_behavior_diff_effs_24-01-22_day3_with_nets_groupA.pkl'
df_file_B_22 = '/home/kai/Documents/Projects/HumanObserveBetEfficacyAnalysis/results/behavior/20240527163117_behavior_diff_effs_24-01-22_day3B_with_nets_groupB.pkl'

df_file_A_29 = '/home/kai/Documents/Projects/HumanObserveBetEfficacyAnalysis/results/behavior/20240527164255_behavior_diff_effs_24-01-29_day3_with_nets_groupA.pkl'
df_file_B_29 = '/home/kai/Documents/Projects/HumanObserveBetEfficacyAnalysis/results/behavior/20240527164833_behavior_diff_effs_24-01-29_day3B_with_nets_groupB.pkl'

df_A = pd.read_pickle(df_file_A_22)
df_B = pd.read_pickle(df_file_B_22) 

df_A_29 = pd.read_pickle(df_file_A_29)
df_B_29 = pd.read_pickle(df_file_B_29)

df_A = pd.concat([df_A, df_A_29])
df_B = pd.concat([df_B, df_B_29])

## only keep the rows in df_A that match an index in df (keeping in mind there might be missing keys)
df_A = df_A[df_A.index.isin(df.index)]
df_B = df_B[df_B.index.isin(df.index)]

In [ ]:
from settings_anal import levc_human_ape_models as ape_models
from settings_anal import levc_human_control_models as control_models
from human_utils_project import sort_train_test


test_liks_A_ape = []
test_liks_A_noape = []
test_liks_B_ape = []
test_liks_B_noape = []

for model in ape_models:
    train_liks_A, test_liks_A = sort_train_test(df_A['step_l_%d' %model], df_A['effs'], test_start=5)
    test_liks_A_ape.append(test_liks_A)

for model in control_models:
    train_liks_A, test_liks_A = sort_train_test(df_A['step_l_%d' %model], df_A['effs'], test_start=5)
    test_liks_A_noape.append(test_liks_A)

for model in ape_models:
    train_liks_B, test_liks_B = sort_train_test(df_B['step_l_%d' %model], df_B['effs'], test_start=4)
    test_liks_B_ape.append(test_liks_B)

for model in control_models:
    train_liks_B, test_liks_B = sort_train_test(df_B['step_l_%d' %model], df_B['effs'], test_start=4)
    test_liks_B_noape.append(test_liks_B)

test_liks_A_ape = np.array(test_liks_A_ape)
test_liks_A_noape = np.array(test_liks_A_noape)
test_liks_B_ape = np.array(test_liks_B_ape)
test_liks_B_noape = np.array(test_liks_B_noape)

test_log_liks_A_ape = np.log(test_liks_A_ape).sum(axis=(2,3)).mean(axis=0)
test_log_liks_A_noape = np.log(test_liks_A_noape).sum(axis=(2,3)).mean(axis=0)
test_log_liks_B_ape = np.log(test_liks_B_ape).sum(axis=(2,3)).mean(axis=0)
test_log_liks_B_noape = np.log(test_liks_B_noape).sum(axis=(2,3)).mean(axis=0)

test_log_liks_ape = np.concatenate([test_log_liks_A_ape, test_log_liks_B_ape])
test_log_liks_noape = np.concatenate([test_log_liks_A_noape, test_log_liks_B_noape])

# test_liks_ape = [test_liks_A_ape[:,:,-1].mean(axis=(0,-1)), test_liks_B_ape[:,:,2].mean(axis=(0,-1)), test_liks_A_ape[:,:,0].mean(axis=(0,-1))]
# test_liks_noape = [test_liks_A_noape[:,:,-1].mean(axis=(0,-1)), test_liks_B_noape[:,:,2].mean(axis=(0,-1)), test_liks_A_noape[:,:,0].mean(axis=(0,-1))]
# plotted_levels = [1, 0.5, 0]

In [ ]:
test_log_liks_ape

array([ -197.83769,  -284.6319 ,  -233.1812 ,  -220.8844 ,  -221.2203 ,
        -733.79504,  -398.6971 ,  -202.44339,  -806.04236,  -345.07184,
        -265.52676,  -203.70082,  -294.41754, -1049.3402 ,  -498.6441 ,
        -972.40173,  -460.56308,  -432.02695,  -192.05376,  -262.62842,
        -236.20499,  -422.20792,  -238.02426,  -196.30826,  -304.10147,
        -314.53888,  -237.30725,  -262.80548,  -312.25357,  -582.5339 ,
        -270.95837,  -709.74207,  -248.02237,  -254.21634,  -317.00272,
        -235.64572,  -870.8379 ,  -445.5019 ,  -567.8578 ,  -620.9753 ,
        -423.09247,  -348.28445,  -378.2017 ,  -357.99625,  -207.94804,
        -589.31476, -1000.8417 ,  -239.45393, -1029.8069 ,  -271.90118,
        -414.3558 ,  -390.80472,  -533.6897 ,  -319.40985,  -360.9077 ,
        -657.719  ,  -338.20792,  -378.60736,  -403.79926, -1011.6603 ,
        -366.1418 ,  -652.0687 ,  -525.0365 , -1397.2034 ,  -316.46332,
        -932.3524 ,  -380.83554,  -250.42998,  -399.5611 ,  -509

In [ ]:
from groupBMC import GroupBMC

L = np.array([test_log_liks_ape, test_log_liks_noape])
result = GroupBMC(L).get_result()

print("Model frequencies", result.frequency_mean)
print("Model frequencies variance", result.frequency_var)
print("Model frequences std", np.sqrt(result.frequency_var))
print("Model SEM", np.sqrt(result.frequency_var)/ np.sqrt(len(test_log_liks_ape)))
print("N=", len(test_log_liks_ape))
print("Exceedance probabilities", result.exceedance_probability)

Model frequencies [0.99553571 0.00446429]
Model frequencies variance [3.93305829e-05 3.93305829e-05]
Model frequences std [0.00627141 0.00627141]
Model SEM [0.00059526 0.00059526]
N= 111
Exceedance probabilities [1.00000000e+00 2.16065456e-37]


In [ ]:
from scipy.stats import ttest_rel
from statsmodels.stats.multitest import multipletests


## MEANS AND SEMS
print("Overall N", len(df))

p_values = []

for level, liks, liks_noape in zip(plotted_levels, test_liks_ape, test_liks_noape):
    ## AVERAGE STATS
    print("Test Controllability ", level)
    print("Mean APE", liks.mean())
    print("StdErr APE", liks.std()/np.sqrt(liks.shape[0]))
    print("N APE", liks.shape[0])
    
    print("Mean NOAPE", liks_noape.mean())
    print("StdErr NOAPE", liks_noape.std()/np.sqrt(liks_noape.shape[0]))
    print("N NOAPE", liks_noape.shape[0])

    ## paired t-test
    ttest = ttest_rel(liks, liks_noape, alternative='greater')
    print(ttest)
    p_values.append(ttest.pvalue)

## MULTIPLE COMPARISONS
rejects, p_values_corrected, _, _ = multipletests(p_values, alpha=0.05, method='holm')
print(rejects)
print(p_values_corrected)

Overall N 111
Test Controllability  1
Mean APE 0.6610438
StdErr APE 0.029985651435308458
N APE 46
Mean NOAPE 0.59373236
StdErr NOAPE 0.0244556605887658
N NOAPE 46
TtestResult(statistic=5.935950362367477, pvalue=1.943115681870144e-07, df=45)
Test Controllability  0.5
Mean APE 0.5539967
StdErr APE 0.03104156052417598
N APE 65
Mean NOAPE 0.51043034
StdErr NOAPE 0.024744829064261845
N NOAPE 65
TtestResult(statistic=4.44496127641961, pvalue=1.7844602690548575e-05, df=64)
Test Controllability  0
Mean APE 0.5938757
StdErr APE 0.0280230078106912
N APE 46
Mean NOAPE 0.5353945
StdErr NOAPE 0.02440152731722509
N NOAPE 46
TtestResult(statistic=5.387383614311002, pvalue=1.2539182859163282e-06, df=45)
[ True  True  True]
[5.82934705e-07 1.78446027e-05 2.50783657e-06]


In [ ]:
import pandas as pd

# Initialize a list to store the results
results = []

for level, liks, liks_noape in zip(plotted_levels, test_liks_ape, test_liks_noape):
    # Calculate statistics
    mean_ape = liks.mean()
    stderr_ape = liks.std()/np.sqrt(liks.shape[0])
    n_ape = liks.shape[0]
    
    mean_noape = liks_noape.mean()
    stderr_noape = liks_noape.std()/np.sqrt(liks_noape.shape[0])
    n_noape = liks_noape.shape[0]

    # Perform t-test
    ttest = ttest_rel(liks, liks_noape, alternative='greater')

    # Calculate degrees of freedom
    df = len(liks) - 1

    # Append results to the list
    results.append([level, n_ape, mean_ape, stderr_ape, mean_noape, stderr_noape, df, ttest.pvalue, ])

# Convert the list to a DataFrame
df_results = pd.DataFrame(results, columns=['Controllability', 'N', 'Mean APE', 'StdErr APE', 'Mean Standard', 'StdErr Standard', 'df', 'p-value',])

# Perform multiple comparisons correction
rejects, p_values_corrected, _, _ = multipletests(df_results['p-value'], alpha=0.05, method='holm')

# Add the corrected p-values and rejection decisions to the DataFrame
df_results['p-value corrected'] = p_values_corrected
df_results['reject'] = rejects

# Set "Controllability" to index column
#df_results.set_index('Controllability', inplace=True, drop=True)
df_results.rename_axis('Index', inplace=True)

df_results = df_results.round(3)

df_results['p-value'] = df_results['p-value'].map('{:.3f}'.format)
df_results['p-value corrected'] = df_results['p-value corrected'].map('{:.3f}'.format)

# Print the DataFrame
df_results

,Controllability,N,Mean APE,StdErr APE,Mean Standard,StdErr Standard,df,p-value,p-value corrected,reject
Index,,,,,,,,,,
0,1.0,46,0.661,0.030,0.594,0.024,45,0.000,0.000,True
1,0.5,65,0.554,0.031,0.510,0.025,64,0.000,0.000,True
2,0.0,46,0.594,0.028,0.535,0.024,45,0.000,0.000,True


In [ ]:
import pandas as pd

# Initialize a list to store the results
results = []

#for level, liks, liks_noape in zip(effs, test_liks_ape, test_liks_noape):
for eff in np.arange(0, 1.01, 0.125):
    print(eff, effs_test[0], effs_test[1])
    if eff in effs_test[0]:
        print(effs_test[0].index(eff))
    else:
        print(effs_test[1].index(eff))
    liks = test_liks_A_ape[:, :, effs_test[0].index(eff)] if eff in effs_test[0] else test_liks_B_ape[:, :, effs_test[1].index(eff)]
    liks = liks.mean(axis=(0,-1))
    liks_noape = test_liks_A_noape[:, :, effs_test[0].index(eff)] if eff in effs_test[0] else test_liks_B_noape[:, :, effs_test[1].index(eff)]
    liks_noape = liks_noape.mean(axis=(0,-1))

    # Calculate statistics
    mean_ape = liks.mean()
    stderr_ape = liks.std()/np.sqrt(liks.shape[0])
    n_ape = liks.shape[0]
    
    mean_noape = liks_noape.mean()
    stderr_noape = liks_noape.std()/np.sqrt(liks_noape.shape[0])
    n_noape = liks_noape.shape[0]

    # Perform t-test
    ttest = ttest_rel(liks, liks_noape, alternative='greater')
    print(ttest)

    # Calculate degrees of freedom
    df = len(liks) - 1

    # Append results to the list
    results.append([level, n_ape, mean_ape, stderr_ape, mean_noape, stderr_noape, df, ttest.pvalue, ])

# Convert the list to a DataFrame
df_results = pd.DataFrame(results, columns=['Controllability', 'N', 'Mean APE', 'StdErr APE', 'Mean Standard', 'StdErr Standard', 'df', 'p-value',])

# Perform multiple comparisons correction
rejects, p_values_corrected, _, _ = multipletests(df_results['p-value'], alpha=0.05, method='holm')

# Add the corrected p-values and rejection decisions to the DataFrame
df_results['p-value corrected'] = p_values_corrected
df_results['reject'] = rejects

# Set "Controllability" to index column
#df_results.set_index('Controllability', inplace=True, drop=True)
df_results.rename_axis('Index', inplace=True)

df_results = df_results.round(3)

# Print the DataFrame
df_results

0.0 [0, 0.25, 0.75, 1.0] [0.125, 0.375, 0.5, 0.625, 0.875]
0
TtestResult(statistic=5.387383614311002, pvalue=1.2539182859163282e-06, df=45)
0.125 [0, 0.25, 0.75, 1.0] [0.125, 0.375, 0.5, 0.625, 0.875]
0
TtestResult(statistic=4.2435791080913665, pvalue=3.620496584365552e-05, df=64)
0.25 [0, 0.25, 0.75, 1.0] [0.125, 0.375, 0.5, 0.625, 0.875]
1
TtestResult(statistic=5.657191703368873, pvalue=5.024909930712447e-07, df=45)
0.375 [0, 0.25, 0.75, 1.0] [0.125, 0.375, 0.5, 0.625, 0.875]
1
TtestResult(statistic=4.300768573277472, pvalue=2.9660742886021207e-05, df=64)
0.5 [0, 0.25, 0.75, 1.0] [0.125, 0.375, 0.5, 0.625, 0.875]
2
TtestResult(statistic=4.44496127641961, pvalue=1.7844602690548575e-05, df=64)
0.625 [0, 0.25, 0.75, 1.0] [0.125, 0.375, 0.5, 0.625, 0.875]
3
TtestResult(statistic=5.22771002563792, pvalue=1.0049307742633908e-06, df=64)
0.75 [0, 0.25, 0.75, 1.0] [0.125, 0.375, 0.5, 0.625, 0.875]
2
TtestResult(statistic=5.500449162942592, pvalue=8.55403202875173e-07, df=45)
0.875 [0, 0.25, 0

,Controllability,N,Mean APE,StdErr APE,Mean Standard,StdErr Standard,df,p-value,p-value corrected,reject
Index,,,,,,,,,,
0,0,46,0.594,0.028,0.535,0.024,45,0.000,0.000,True
1,0,65,0.537,0.029,0.501,0.024,64,0.000,0.000,True
2,0,46,0.606,0.031,0.546,0.027,45,0.000,0.000,True
3,0,65,0.565,0.030,0.525,0.025,64,0.000,0.000,True
4,0,65,0.554,0.031,0.510,0.025,64,0.000,0.000,True
5,0,65,0.573,0.032,0.523,0.026,64,0.000,0.000,True
6,0,46,0.664,0.029,0.607,0.024,45,0.000,0.000,True
7,0,65,0.576,0.032,0.535,0.024,64,0.001,0.001,True
8,0,46,0.661,0.030,0.594,0.024,45,0.000,0.000,True
